In [2]:
pip install json_lines


In [3]:
import json_lines 
import pandas as pd
import re
import string
%matplotlib inline

In [38]:
X=[]; y=[]; z=[]
with open('review.jl', 'rb') as f: 
  for item in json_lines.reader(f):
    if (item != None and item != '') and 'reviewText' in item and len(item['reviewText'].split(" "))>=50:
      X.append(item['asin']) 
      y.append(item['reviewText'])
      z.append(item['overall'])
      

In [40]:
data = {'ID': X,'Rating':z,'Review': y}
df = pd.DataFrame(data=data)
print(df)

                ID  Rating                                             Review
0       1118461304     5.0  Not one thing in this book seemed an obvious o...
1       1118461304     5.0  I have enjoyed Dr. Alan Gregerman's weekly blo...
2       1118461304     5.0  Alan Gregerman believes that innovation comes ...
3       1118461304     5.0  Alan Gregerman is a smart, funny, entertaining...
4       1118461304     5.0  As I began to read this book, I was again remi...
...            ...     ...                                                ...
112867  B01HJ6OD0A     3.0  I had to rate this a just maybe it's ok..becau...
112868  B01HJH0KLK     5.0  The original one lasted about 3 years, then I ...
112869  B01HJH397M     5.0  The pump arrived right on time and very easy t...
112870  B01HJH2PY0     4.0  This replacement part fit but it came in stain...
112871  B01HJH92JQ     3.0  The switch works with my dishwasher after I mo...

[112872 rows x 3 columns]


In [41]:
def cleaning(text):
  text=text.lower()
  text=re.sub('\[.*?\],!','', text)
  text=re.sub('[%s]'% re.escape(string.punctuation),'',text)
  text=re.sub('\w*\d\w*','',text)
  return text

cleaned = lambda x: cleaning(x)

In [42]:
def cleaning2(text):
  text=text.lower()
  text=re.sub('[''""...]','', text)
  text=re.sub('\n','',text)
  return text

cleaned2 = lambda x: cleaning2(x)

In [45]:
df['Review']=pd.DataFrame(df.Review.apply(cleaned))
df['Review']=pd.DataFrame(df.Review.apply(cleaned2))
print(df)

                ID  Rating                                             Review
0       1118461304     5.0  not one thing in this book seemed an obvious o...
1       1118461304     5.0  i have enjoyed dr alan gregermans weekly blog ...
2       1118461304     5.0  alan gregerman believes that innovation comes ...
3       1118461304     5.0  alan gregerman is a smart funny entertaining w...
4       1118461304     5.0  as i began to read this book i was again remin...
...            ...     ...                                                ...
112867  B01HJ6OD0A     3.0  i had to rate this a just maybe its okbecause ...
112868  B01HJH0KLK     5.0  the original one lasted about  years then i re...
112869  B01HJH397M     5.0  the pump arrived right on time and very easy t...
112870  B01HJH2PY0     4.0  this replacement part fit but it came in stain...
112871  B01HJH92JQ     3.0  the switch works with my dishwasher after i mo...

[112872 rows x 3 columns]


In [69]:
a=[]; b=[]; c=[]
with open('product.jl', 'rb') as file: 
  for item in json_lines.reader(file):
    if (item != None and item != '') and 'title' in item and 'price' in item and item['price']!='' and len(item['price'])<=8:
      a.append(item['asin']) 
      b.append(item['title'])
      # c.append(item['price'])
      price = item['price']
      price = price.replace("$", "").strip()
      price = price.replace(",", "").strip()
      price = price.replace(".a", "").strip()
      if (price.find('-') == -1):
         c.append(price)
      else:
         priceRange = [float(p.strip()) for p in price.split("-")]
         price = sum(priceRange) / len(priceRange)
         c.append(price)

In [71]:
pro = {'ID': a,'Product':b, 'Price':c}
df_pro = pd.DataFrame(data=pro)
df_pro['Product']=pd.DataFrame(df_pro.Product.apply(cleaned))
df_pro['Product']=pd.DataFrame(df_pro.Product.apply(cleaned2))
print(df_pro)

               ID                                            Product   Price
0      7861850250   x tupperware pure amp fresh unique covered co...    3.62
1      8792559360                     the cigar  moments of pleasure  150.26
2      B00002N5EL      eaton wiring  arrow hart  chrome power outlet    3.43
3      B00002N7HY  leviton   electrical receptacle  vac  a  pole ...    6.98
4      B00002N9OE              amerock   diameter ceramic knob black    7.68
...           ...                                                ...     ...
19959  B01HJH92JQ                                      bosch  switch   40.23
19960  B01HJH2I5G                                  bosch  panelfacia   95.99
19961  B01HJH0J1G                                   bosch  ice maker  250.79
19962  B01HJHHEA0                                   frigidaire  knob   14.99
19963  B01HJH397M                                   bosch  pumpdrain   64.99

[19964 rows x 3 columns]


In [72]:
result = pd.merge(df,df_pro[['ID','Product','Price']],on='ID')
print(list(result))
print(result)

['ID', 'Rating', 'Review', 'Product', 'Price']
               ID  ...   Price
0      B00004SQHD  ...    1.41
1      B00004SQHD  ...    1.41
2      B00004SQHD  ...    1.41
3      B00004SQHD  ...    1.41
4      B00004SQHD  ...    1.41
...           ...  ...     ...
76817  B01HICGO1G  ...   17.00
76818  B01HJH0KLK  ...   78.22
76819  B01HJH397M  ...   64.99
76820  B01HJH2PY0  ...  101.13
76821  B01HJH92JQ  ...   40.23

[76822 rows x 5 columns]


In [ ]:
from google.colab import files
result.to_csv('result.csv') 
files.download('result.csv')
